# Start of DS5559 Final Project

Team Left Twix Members

* Alice Wright - aew7j
* Edward Thompson - ejt8b
* Michael Davies -  mld9s
* Sam Parsons - sp8hp

In STAT 6021 members of our cohort looked at Transportation Network Company data sets to see if there was a potential relationship between tipping and other indicators, specifically with “transportation network providers” i.e. rideshares such as Uber, Lyft, etc.  At that point in our Data Science journey we did not have the skills or equipment to investigate this question in depth.  

Utilizing machine learning skills from SYS 6018 and applying Spark to this dataset we hope to come up with a more robust set of answers and potentially a better predictor of tipping. With other classification algorithms such as random forest and the heavy-weight data processing of Spark, will we be able to create a more robust predictive model?


Potential Questions from the TNC Data:

* Can it be predicted what fares are most likely to tip the driver?
* Is there a relationship between time of the fare and tipping? (workday stat, bar close, weekday, weekend, etc)
* Is there a relationship between start or end location of the ride and tipping? (downtown pickup, north shore, airport, etc)
* Is there a relationship between length or cost of ride and tipping? (do longer rides result in tips)
* Using this data would we be able to make recommendations to drivers to maximize likelihood of receiving a tip?
* Is the likelihood of tipping changing over time?  Are more rides being tipped?
* Are there re-identification abilities in this dataset? For instance, can we find records for a person who reliably takes a rideshare to/from work every day thereby linking a home address to a work address?




Additionally, joining in additional datasets may yield answers to questions about external factors such as:
* How did news reporting/social media on rideshare companies correlate with tipping?
* What relationship(s) does trip demand have with the stocks of these companies?

Data Source:
The best data source for this appears to be from the City of Chicago, as it is large (169M records and 21 columns), relatively clean, anonymized, and accessible via API.

City of Chicago:
https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips/m6dm-c72p

So far we have only pulled the data down via a CSV.

Code Rubric

* Data Import and PreProcessing | 2 pts

* Data splitting/sampling | 1 pt

* EDA (min two graphs) | 2 pts

* Model construction (min 3 models) | 3 pts

* Model evaluation | 2 pts

In [1]:
# import context manager: SparkSession
from pyspark.sql import SparkSession

# import data types
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType
import pyspark.sql.types as typ
import pyspark.sql.functions as F
import os

from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.sql.types import *

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("mllib_classifier") \
        .config("spark.executor.memory", '21g') \
        .config('spark.executor.cores', '2') \
        .config('spark.executor.instances', '3') \
        .config("spark.driver.memory",'1g') \
        .getOrCreate()
sc = spark.sparkContext

# import data manipulation methods
from pyspark.ml.feature import Binarizer
from pyspark.ml import Pipeline  
from pyspark.ml.feature import *  
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier

from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import VectorAssembler 
from pyspark.mllib.linalg import Vectors

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [2]:
%whos

Variable                            Type            Data/Info
-------------------------------------------------------------
ArrayType                           type            <class 'pyspark.sql.types.ArrayType'>
Binarizer                           type            <class 'pyspark.ml.feature.Binarizer'>
BinaryType                          type            <class 'pyspark.sql.types.BinaryType'>
BooleanType                         type            <class 'pyspark.sql.types.BooleanType'>
BucketedRandomProjectionLSH         type            <class 'pyspark.ml.featur<...>etedRandomProjectionLSH'>
BucketedRandomProjectionLSHModel    type            <class 'pyspark.ml.featur<...>andomProjectionLSHModel'>
Bucketizer                          type            <class 'pyspark.ml.feature.Bucketizer'>
ByteType                            type            <class 'pyspark.sql.types.ByteType'>
ChiSqSelector                       type            <class 'pyspark.ml.feature.ChiSqSelector'>
ChiSqSelectorModel  

In [3]:
#clear old df
#del (df)

# Read in our Dataset

## Create a Custom Schema.  
This schema was been primarly determined by using a much smaller dataset and letting spark infer the schema.  We encountered an issue with spark reading in the ENTIRE dataset as NULL when there was a type mismatch.  Only the data we are likely to use later has been assigned to a specific type, otherwise it is left as a string type.

In [4]:
# create a custom schema.  

customSchema = StructType([
    StructField('Trip_ID', StringType(), True),        
    StructField('Trip_Start_Timestamp', StringType(), True),
    StructField('Trip_End_Timestamp', StringType(), True),
    StructField('Trip_Seconds', DoubleType(), True),
    StructField('Trip_Miles', DoubleType(), True),
    StructField('Pickup_Census_Tract', StringType(), True),
    StructField('Dropoff_Census_Tract', StringType(), True),
    StructField('Pickup_Community_Area', DoubleType(), True),
    StructField('Dropoff_Community_Area', DoubleType(), True),
    StructField("Fare", DoubleType(), True),
    StructField("Tip", DoubleType(), True),
    StructField("Additional_Charges", DoubleType(), True),
    StructField("Trip_Total", StringType(), True),
    StructField("Shared_Trip_Authorized", BooleanType(), True),
    StructField("Trips_Pooled", DoubleType(), True),
    StructField('Pickup_Centroid_Latitude', StringType(), True),
    StructField('Pickup_Centroid_Longitude', StringType(), True),
    StructField('Pickup_Centroid_Location', StringType(), True),
    StructField('Dropoff_Centroid_Latitude', StringType(), True),
    StructField('Dropoff_Centroid_Longitude', StringType(), True),
    StructField('Dropoff_Centroid_Location', StringType(), True)
])

#old readin.  Infer is slow for large dataset
#df = spark.read.csv('/../../project/ds5559/Alice_Ed_Michael_Sam_project/BigTrips.csv', header = True, inferSchema=True)

#read in the data to a dataframe
df = spark.read.csv('/../../project/ds5559/Alice_Ed_Michael_Sam_project/BigTrips.csv', header = True, schema=customSchema)
df.show(5)

+--------------------+--------------------+--------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+------------------------+-------------------------+------------------------+-------------------------+--------------------------+-------------------------+
|             Trip_ID|Trip_Start_Timestamp|  Trip_End_Timestamp|Trip_Seconds|Trip_Miles|Pickup_Census_Tract|Dropoff_Census_Tract|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|Pickup_Centroid_Latitude|Pickup_Centroid_Longitude|Pickup_Centroid_Location|Dropoff_Centroid_Latitude|Dropoff_Centroid_Longitude|Dropoff_Centroid_Location|
+--------------------+--------------------+--------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+---+-------

In [5]:
#Doesn't update if you don't resave the variable

df = df.drop('Trip_End_Timestamp', 
             'Pickup_Census_Tract',
             'Dropoff_Census_Tract',
             'Pickup_Centroid_Latitude',
             'Pickup_Centroid_Longitude', 
             'Pickup_Centroid_Location', 
             'Dropoff_Centroid_Latitude', 
             'Dropoff_Centroid_Longitude', 
             'Dropoff_Centroid_Location')

In [6]:
df.printSchema()

root
 |-- Trip_ID: string (nullable = true)
 |-- Trip_Start_Timestamp: string (nullable = true)
 |-- Trip_Seconds: double (nullable = true)
 |-- Trip_Miles: double (nullable = true)
 |-- Pickup_Community_Area: double (nullable = true)
 |-- Dropoff_Community_Area: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tip: double (nullable = true)
 |-- Additional_Charges: double (nullable = true)
 |-- Trip_Total: string (nullable = true)
 |-- Shared_Trip_Authorized: boolean (nullable = true)
 |-- Trips_Pooled: double (nullable = true)



In [7]:
df2 = df.sample(False, .005, seed = 2021) #increased our sample size

In [8]:
df2.count()

244762

In [9]:
#delete the big df for now
del (df)

#hopefully that will make things faster 

In [10]:
#fill our NA community areas

df2 = df2.na.fill(value=78,subset=['Pickup_Community_Area', 'Dropoff_Community_Area'])

In [11]:
# make a binary tip/no tip indicator
# https://spark.apache.org/docs/2.2.0/ml-features.html#binarizer

binarizer = Binarizer(threshold=0, inputCol="Tip", outputCol="binarized_tip")
df2 = binarizer.transform(df2)

In [12]:
df2.cache()

DataFrame[Trip_ID: string, Trip_Start_Timestamp: string, Trip_Seconds: double, Trip_Miles: double, Pickup_Community_Area: double, Dropoff_Community_Area: double, Fare: double, Tip: double, Additional_Charges: double, Trip_Total: string, Shared_Trip_Authorized: boolean, Trips_Pooled: double, binarized_tip: double]

In [13]:
# split the data

# our model didn't work on the standard test train split.  Prof. Tashman recomended upscalling the help with the imbalanced dataset.
#from https://spark.apache.org/docs/2.1.0/ml-tuning.html#train-validation-split

train_inital, test = df2.randomSplit([0.8, 0.2], seed=2021)

# cahce our test values for later speed
test.cache()

# oversampleing code sample
# https://stackoverflow.com/questions/53273133/how-to-perform-up-sampling-using-sample-functionpy-spark

df_a = train_inital.filter(train_inital['binarized_tip'] == 0)
df_b = train_inital.filter(train_inital['binarized_tip'] == 1)

org_a_count = df_a.count()
org_b_count = df_b.count() 


ratio = df_a.count() / df_b.count()
# print(ratio)

df_b_overampled = df_b.sample(withReplacement=True, fraction=ratio, seed=2021)

# cahce our train values for later speed
train = df_a.unionAll(df_b_overampled).cache()

df_af = train.filter(train_inital['binarized_tip'] == 0)
df_bf = train.filter(train_inital['binarized_tip'] == 1)
fin_a_count = df_af.count()
fin_b_count = df_bf.count() 

print("Original No Tip Count: ", org_a_count)
print("Original Tip Count   : ", org_b_count)
print("")
print("Final No Tip Count   : ", fin_a_count)
print("Final Tip Count      : ", fin_b_count)


Original No Tip Count:  163097
Original Tip Count   :  32843

Final No Tip Count   :  163097
Final Tip Count      :  163589


In [14]:
test.show(5)
train.show(5)

+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+
|             Trip_ID|Trip_Start_Timestamp|Trip_Seconds|Trip_Miles|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|binarized_tip|
+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+
|003ca5c025410fcc9...|12/01/2019 11:15:...|       739.0|       4.7|                  6.0|                  32.0|10.0|0.0|              2.55|     12.55|                 false|         1.0|          0.0|
|00bcab98504c1e0c9...|12/01/2019 08:00:...|      1414.0|       6.7|                  6.0|                  32.0|15.0|0.0|              2.55|     17.55|                 false|         1.0|     

Our training size has increased.  This is to be expected in upscaling.

Good reference: https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets

In [15]:
#just as a reminder what was the truth in our test data?

dft_a = test.filter(train_inital['binarized_tip'] == 0)
dft_b = test.filter(train_inital['binarized_tip'] == 1)
count_test_a = dft_a.count()
count_test_b = dft_b.count()
print(count_test_a)
print(count_test_b)


40467
8355


In [16]:
#del (df2)

# Pipeline for LR

In [17]:
def cmacc(pred):
    # make a confusion matrix and return the accuracy
    # select predictions and labels from prediction transform as rdd as there isn't a DF function for this
    pred_rdd= pred.select('prediction').rdd.flatMap(lambda x: x)
    label_rdd = pred.select('binarized_tip').rdd.flatMap(lambda x: x)
    
    #zip them together
    predictionAndLabels =  pred_rdd.zip(label_rdd)
    
    #metrics transform
    metrics = MulticlassMetrics(predictionAndLabels)
    
    #make our confusion matrix
    cm = metrics.confusionMatrix().toArray()

    #calc accuracy from confusion matrix
    
    acc = (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
    
    print(cm)
    print()
    print(acc)

In [18]:
# pipeline steps for Logistic Regression:

#onehotencoder to pickup
ohe_pu = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")

#onehotencoder to dropoff
ohe_do = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")

#assemble the vector or LR

# our colulms for lr
predictor_col_for_lr = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec']

lr_va = VectorAssembler(inputCols=predictor_col_for_lr, outputCol="features") 

#scale our LR

lr_scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

#what do we want to do if we are doing a parameter search? make the parameters as variables and just do a loop?
#we learned that this week.  May also need to add in cv step

lr = LogisticRegression(maxIter=10,
                        regParam=0.1,
                        elasticNetParam=0.3,
                        featuresCol="features",
                        labelCol="binarized_tip")

# Build the pipeline
lr_pipeline = Pipeline(stages=[ohe_pu, ohe_do, lr_va, lr_scaler, lr])

# Fit the pipeline
lr_model = lr_pipeline.fit(train)

# Make a prediction
lr_prediction = lr_model.transform(test)

In [19]:
cmacc(lr_prediction)

[[29341. 11126.]
 [ 4918.  3437.]]

0.671377657613371


# Pipeline for RF

In [22]:
# pipeline steps for Random Forest:

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="binarized_tip", outputCol="indexedLabel")#.fit(df2)

#onehotencoder to pickup
ohe_pu = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")

#onehotencoder to dropoff
ohe_do = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")

# our colulms for rf
predictor_col_for_rf = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec']

rf_va = VectorAssembler(inputCols=predictor_col_for_rf, outputCol="features") 

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)#.fit(df2)

#skip scaling for now

# lr_scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
#                         withStd=True, withMean=False)

#what do we want to do if we are doing a parameter search? make the parameters as variables and just do a loop?
#we learned that this week.  May also need to add in cv step

rf = RandomForestClassifier(labelCol="indexedLabel", 
                            featuresCol="indexedFeatures", 
                            numTrees=10)

# Build the pipeline
rf_pipeline = Pipeline(stages=[ohe_pu, ohe_do, labelIndexer, rf_va, featureIndexer, rf])

# Fit the pipeline
rf_model = rf_pipeline.fit(train)

# Make a prediction
rf_prediction = rf_model.transform(test)

# Select example rows to display.
rf_prediction.select("prediction", "binarized_tip","features").show(5) 

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="binarized_tip", 
    predictionCol="prediction", 
    metricName="accuracy")

accuracy = evaluator.evaluate(rf_prediction)

print("Test Error = %g" % (1.0 - accuracy))

rfModel = rf_model.stages[5]
print(rfModel)  # summary only

+----------+-------------+--------------------+
|prediction|binarized_tip|            features|
+----------+-------------+--------------------+
|       1.0|          0.0|(162,[0,1,2,3,5,1...|
|       1.0|          0.0|(162,[0,1,2,3,5,1...|
|       1.0|          0.0|(162,[0,1,2,3,5,3...|
|       1.0|          0.0|(162,[0,1,2,3,5,3...|
|       1.0|          1.0|(162,[0,1,2,3,5,1...|
+----------+-------------+--------------------+
only showing top 5 rows

Test Error = 0.662099
RandomForestClassificationModel (uid=RandomForestClassifier_5ea99b4e5245) with 10 trees


In [23]:
cmacc(rf_prediction)

[[11876. 28591.]
 [ 3734.  4621.]]

0.3379009462947032


# Pipeline for GBT

In [24]:
#try a GBT 

# https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-classifier

In [25]:
#onehotencoder to pickup
ohe_pu = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")

#onehotencoder to dropoff
ohe_do = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="binarized_tip", outputCol="indexedLabel")#.fit(data)

#assemble the vector or GBT

# our colulms for gbt
predictor_col_for_gbt = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec']

gbt_va = VectorAssembler(inputCols=predictor_col_for_rf, outputCol="features") 

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)#.fit(data)

# Train a GBT model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=5)

# Chain indexers and GBT in a Pipeline
gbt_pipeline = Pipeline(stages=[ohe_pu, ohe_do, labelIndexer, gbt_va, featureIndexer, gbt])

# Train model.  This also runs the indexers.
gbt_model = gbt_pipeline.fit(train)

# Make predictions.
gbt_prediction = gbt_model.transform(test)

gbt_prediction.show(3)

# Select example rows to display.
gbt_prediction.select("prediction", "binarized_tip", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="binarized_tip", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(gbt_prediction)
print("Test Error = %g" % (1.0 - accuracy))
gbtModel = gbt_model.stages[5]
print(gbtModel)  # summary only

+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+-------------------------+--------------------------+------------+--------------------+--------------------+--------------------+--------------------+----------+
|             Trip_ID|Trip_Start_Timestamp|Trip_Seconds|Trip_Miles|Pickup_Community_Area|Dropoff_Community_Area|Fare|Tip|Additional_Charges|Trip_Total|Shared_Trip_Authorized|Trips_Pooled|binarized_tip|Pickup_Community_Area_vec|Dropoff_Community_Area_vec|indexedLabel|            features|     indexedFeatures|       rawPrediction|         probability|prediction|
+--------------------+--------------------+------------+----------+---------------------+----------------------+----+---+------------------+----------+----------------------+------------+-------------+-------------------------+--------------------------+------------+-------

In [26]:
cmacc(gbt_prediction)

[[14237. 26230.]
 [ 4404.  3951.]]

0.37253697103764694
